# Übung 3

## 2) Textklassifikation mit multinomialen Bayesklassifikator

## 2a) Herunterladen und Entpacken der Datenbank

In [43]:
import pandas as pd
import numpy as np
import matplotlib as mp
import tarfile
import os.path
import urllib.request
import skimage
import scipy as sp
from random import shuffle

In [44]:
filename = "20news-18828.tar.gz"
url = "http://qwone.com/~jason/20Newsgroups/" + filename
if not os.path.isfile("./" + filename):
    urllib.request.urlretrieve(url, filename)
    dateDownloaded = !date
    print(dateDownloaded)

In [45]:
tar = tarfile.open("./" + filename)
tar.extractall()
tar.close()
foldername = filename.replace(".tar.gz", "")

## 2b) ...

In [46]:
import codecs
newsgroup_names = ["alt.atheism", "comp.graphics", "sci.space", "talk.religion.misc"]
newsgroups = {}
for newsgroup_name in newsgroup_names:
    newsgroup_folder = foldername+"/"+newsgroup_name
    text_names = os.listdir(newsgroup_folder)
    for text_name in text_names:
        with open(newsgroup_folder+"/"+text_name, "r",encoding='ISO-8859-1') as f:
            text = f.read()
            newsgroups.update({text_name : [newsgroup_name, text]})
newsgroups
len(newsgroups)

3387

In [47]:
for newsgroup_key in newsgroups:
    not_used, not_used, stripped_text = newsgroups[newsgroup_key][1].partition('\n\n')
    newsgroups.update({newsgroup_key : [newsgroups[newsgroup_key][0],stripped_text]})
newsgroups
len(newsgroups)

3387

## 2c)

In [48]:
import re
word_count_newsgroups = {}
for newsgroup_key in newsgroups:
    splitted_text = re.compile(r"(?u)\b\w\w+\b").findall(newsgroups[newsgroup_key][1].lower())
    word_count = {i:splitted_text.count(i) for i in splitted_text}
    word_count_newsgroups.update({newsgroup_key : word_count})

In [50]:
word_count_all = {}
for word_count_newsgroups_key in word_count_newsgroups:
    for single_word_count_key in newsgroups_splitted[word_count_newsgroups_key]:
        #print(single_word_count_key, newsgroups_splitted[word_count_newsgroups_key][single_word_count_key] )
        if single_word_count_key not in word_count_all:
            word_count_all.update({single_word_count_key : newsgroups_splitted[word_count_newsgroups_key][single_word_count_key]})
        else:
            multiple_word_count = newsgroups_splitted[word_count_newsgroups_key][single_word_count_key] + word_count_all[single_word_count_key]
            word_count_all.update({single_word_count_key : multiple_word_count})
len(word_count_all)

41777

In [57]:
#word_count_newsgroups words per text {newsgroup:{word:count}}
#word_count_classes words per class {class:{word:count}}
#word_count_all all words over all texts {word:count}

## 2d)

In [59]:
def insert_words(word_count_classes, class_key, word_count):
    if class_key not in word_count_classes:
        word_count_classes.update({class_key : word_count})
    else:
        for word in word_count:
            if word not in word_count_classes[class_key]:
                word_count_classes[class_key].update({word : word_count[word]})
            else:
                mulitple_word_count = word_count_classes[class_key][word] + word_count[word]
                word_count_classes[class_key].update({word : multiple_word_count})
    return word_count_classes

In [ ]:
from random import shuffle
newsgroups_keys_shuffled = list(newsgroups.keys())
shuffle(newsgroups_keys_shuffled)
word_count_classes_training = {}
for newsgroup_key in newsgroups_keys_shuffled:
    splitted_text = re.compile(r"(?u)\b\w\w+\b").findall(newsgroups[newsgroup_key][1].lower())
    word_count = {i:splitted_text.count(i) for i in splitted_text}
    class_key = newsgroups[newsgroup_key][0]
    word_count_classes_training=insert_words(word_count_classes_training, class_key, word_count)
                
for class_key in word_count_classes_training:
    print(class_key, len(word_count_classes_training[class_key]))
    
### TODO split in 60% training and 40% test data

In [67]:
#training_data = []
#training_data_newsgroup = []
#test_data = []
#test_data_newsgroup = []
#split_size = int(len(word_count_newsgroups)*0.6)
#
#for word_count_newsgroups_key, index in zip(word_count_newsgroups,range(0,len(word_count_newsgroups))):
#    word_count = np.empty([len( word_count_newsgroups[word_count_newsgroups_key]),2])
#    for word, word_index in zip(word_count_newsgroups[word_count_newsgroups_key],range(0,len(word_count_newsgroups[word_count_newsgroups_key]))):
#        word_count[word_index,0]=word
#        word_count[word_index,1]=word_count_newsgroups[word_count_newsgroups_key][word]      
#                                
#    if index < split_size:
#        training_data.append(word_cound)
#        training_data_newsgroup.append(word_count_newsgroups_key)
#    else:        
#        test_data.append( word_count)
#        test_data_newsgroup.append(word_count_newsgroups_key)

In [68]:
#training_data_mean = training_data.mean(axis=0)
#centered_training_data = training_data.copy()
#centered_training_data=centered_training_data-training_data_mean
#
#U, D, Vt = np.linalg.svd(training_data)
#
#eigenvalues = np.square(D)
#
#centered_test_data = test_data.copy()
#centered_test_data=centered_test_data-training_data_mean
#
#eigen_values_dataframe = pd.DataFrame(eigenvalues[0:150])
#
#%pylab inline
#eigen_values_dataframe.plot(logy=True)
#
#test_images = np.dot(Vt[:7], centered_test_data.T).T
#training_images = np.dot(Vt[:7], centered_training_data.T).T
#
#test_images.shape, training_images.shape

## Konfiguration

In [ ]:
%load_ext version_information
%version_information numpy, pandas, matplotlib, skimage